In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from voc_data_reader import VOCDataReader
from fcn import FCN
import cv2, os, sys
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug('')
%matplotlib inline

In [ ]:
TRAINVAL_ROOT_DIR = '/home/paperspace/PASCAL-VOC-Dataset/TrainVal'
TEST_ROOT_DIR = '/home/paperspace/PASCAL-VOC-Dataset/Test'
VGG_PARAMS_ROOT_DIR = '/home/paperspace/FCN/vgg-weights'

fcn = FCN(TRAINVAL_ROOT_DIR, TEST_ROOT_DIR, VGG_PARAMS_ROOT_DIR)
dsr = VOCDataReader(TRAINVAL_ROOT_DIR, TEST_ROOT_DIR)

# fcn.train_coarse(1000, 100)

In [ ]:
tb = dsr.next_train_batch(1)
images = tb['batch']
annotations = tb['annotations']
names = tb['names']

img_path = os.path.join(TRAINVAL_ROOT_DIR,'VOCdevkit/VOC2011/JPEGImages/{}.jpg'.format(names[0]))
img_unnormalized = cv2.imread(img_path)[:,:,::-1]

for i in images: img = i
for a in annotations: ann = a
#img = np.reshape(img, [1, img.shape[0], img.shape[1], img.shape[2]])
best_params_ckpt = '.'.join(os.listdir('best_params')[0].split('.')[:-1])
segmentation = fcn.infer(img, os.path.join('best_params', best_params_ckpt))

def label_matrix_to_image(label_mat, image_file = None):
    def convert(label_mod):
        # logging.debug('trying to hash {}'.format(label))
        label = label_mod[0]
        return np.array(dsr.voc_color_map[label], dtype=np.float32)
    logging.debug('Label mat shape: {}'.format(label_mat.shape))
    label_mat[label_mat == 21] = 255
    np.set_printoptions(threshold=np.nan)
    # logging.debug('{}'.format(label_mat))
    label_mat = np.expand_dims(label_mat, axis=2)
    label_mat = np.concatenate([label_mat, np.zeros(label_mat.shape), np.zeros(label_mat.shape)], axis=2)
    img_mat = np.apply_along_axis(convert, 2, label_mat)
    logging.debug('Converted shape: {}'.format(img_mat.shape))
    if image_file is not None:
        plt.imsave(fname=image_file, arr=img_mat, format='png')
    return img_mat

plt.imshow(label_matrix_to_image(segmentation))

In [ ]:
plt.imshow(label_matrix_to_image(ann))

In [ ]:
plt.imshow(img_unnormalized)